In [1]:
import numpy as np
import random
import scipy.stats as sps
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.8)

In [2]:
# зафиксируем сид для воспроизводимости генерации
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

## Критерий Вальда

Вы провели эксперимент и получили данные из экспоненциального распределения. 

In [4]:
sample = [0.11731702, 0.75253036, 0.32918642, 0.22823564, 0.04240622,
        0.04239907, 0.01495969, 0.50280772, 0.22977054, 0.30781252,
        0.00519983, 0.87588937, 0.44660739, 0.05967191, 0.05016975,
        0.05065286, 0.09068843, 0.18598196, 0.14138427, 0.08605575,
        0.23659272, 0.03755863, 0.08637888, 0.1140693 , 0.15223367,
        0.384484  , 0.05568397, 0.18050729, 0.22437618, 0.01189096]

Вы хотите проверить, является ли это распределение с параметром $\lambda=2$. Используя Критерий Вальда, сделайте вывод по данному предположению.

### Двусторонняя альтернатива
$X_1, ... X_n$ - выборка из распределения $Exp(\theta)$.  
Проверьте гипотезу $\mathsf{H}_0\colon \theta = 2$ vs. $\mathsf{H}_1\colon \theta \neq 2$

Из лекции вы узнали про критерий Вальда.
Для случая двусторонней альтернативы $\mathsf{H}_1\colon \theta \neq \theta_0$ критерий имел следующий вид:
$$\large{S = \left\{ \left|\sqrt{n} \frac{\hat{\theta} - \theta_0}{\hat{\sigma}} \right| > z_{1 - \frac{\alpha}{2}} \right\}}$$

где $\hat{\theta}$ &mdash; асимптотически нормальная оценка $\theta$ с асимптотической дисперсией $\sigma^2(\theta)$, 
$\hat{\sigma}$ &mdash; состоятельная оценка $\sigma(\theta)$.  

Эквивалентный асимптотичсекий доверительный интервал для параметра $\theta$ уровня доверия $1-\alpha$
$$C = \left( \hat{\theta} - \frac{z_{1-\alpha/2} \hat{\sigma}}{\sqrt{n}}, \hat{\theta} + \frac{z_{1-\alpha/2} \hat{\sigma}}{\sqrt{n}}\right)$$

На первой лекции вы получали, что $\frac{1}{\overline{X}}$ &mdash; АНО для параметра $\theta$ c асимптотической дисперсией $\theta^2$

**Выпишите состоятельную оценку дисперсии и статистику критерия Вальда**


**Ответ:**  

Состоятельная оценка для дисперсии:

$$
\frac{1}{\overline{X}^{2}} = \sigma^{2}
$$

Первым шагом необходимо выставить уровень значимости, поставим $\alpha = 0.05$

In [5]:
alpha = 0.05
theta = 2 # тета из основной гипотезы
n = len(sample)

Посчитаем квантиль (критическое значение)

In [6]:
z = sps.norm.ppf(1 - alpha / 2)
z

1.959963984540054

Посчитайте статистику, которую будете сравнивать с критическим значением. Выведите значение полученной статистики.

In [7]:
statistic = np.sqrt(n) * (1 / np.mean(sample) - theta) * np.mean(sample)
statistic

3.2704505437191247

Сравним модуль статистики с критическим значением

In [8]:
np.absolute(statistic) > z

True

**Какой вывод можно сделать?**

**Вывод:** 

Гипотеза опровергнута $\to$ так как значение статистики больше критического $\to$ $\theta \neq 2$

Посчитайте доверительный интервал

In [9]:
left = 1 / np.mean(sample) - sps.norm.ppf(1 - alpha / 2) / (np.sqrt(n) * np.mean(sample))
right = 1 / np.mean(sample) + sps.norm.ppf(1 - alpha / 2) / (np.sqrt(n) * np.mean(sample))
print(f"Доверительный интервал ({left:.2f}, {right:.2f})")

Доверительный интервал (3.19, 6.74)


**Какой вывод можно сделать?**

**Вывод:** 

$\mathfrak{H_{0}} $ отвергнута так как $\theta$ меньше левой границы доверительного интервала

На лекции вы узнали про p-value &mdash; это вероятность получить при справедливости $H_0$ такое значение статистики $t = T(x)$ или еще более экстремальное, то есть в случае двустороннего критерия
$$p(x) = \mathsf{P}_0(T(X) \geq|t|) + \mathsf{P}_0(T(X) \leq -|t|)$$
Посчитайте p-value. Для этого можно использовать функции из библиотеки `scipy.stats`.

In [10]:
p_value = 2 * sps.norm.sf(statistic)
p_value

0.0010737631605120244

Оформите подсчет статистики и  p-value в виде одной функции.

In [11]:
def wald_test_two_sided(sample, theta, estimation_theta, estimation_sigma):
    """
    param sample: реализация выборки
    param theta: истинное значение параметра
    param estimation_theta: оценка параметра
    param estimation_sigma: оценка асимптотической дисперсии оценки estimation_sigma

    return statistic
    return p_value
    return conf_int - доверительный интервал
    """
    n = len(sample)
    statistic = np.sqrt(n) * (estimation_theta - theta) / estimation_sigma
    p_value = 2 * sps.norm.sf(np.abs(statistic))
    left = 1 / np.mean(sample) - sps.norm.ppf(1 - alpha / 2) / (np.sqrt(n) * np.mean(sample))
    right = 1 / np.mean(sample) + sps.norm.ppf(1 - alpha / 2) / (np.sqrt(n) * np.mean(sample))
    result = {'p_value': p_value, 'left': left, 'right': right, 'statistic': statistic}
    return result

### Правосторонняя и левосторонняя альтернативы

In [ ]:
def wald_test(sample, theta, estimation_theta, estimation_sigma, alternative='two_sided'):
    """
    param sample: реализация выборки
    param theta: истинное значение параметра
    param estimation_theta: оценка параметра
    param estimation_sigma: оценка асимптотической дисперсии оценки estimation_sigma
    param alternative: вид альтернативной гипотезы, может принимать одно из значений 'two_sided', 'less', 'greater'

    return statistic
    return p_value
    return conf_int - доверительный интервал
    """
    n = len(sample)
    statistic = np.sqrt(n) * (estimation_theta - theta) / estimation_sigma
    
    if alternative is 'two_sided':
        return wald_test_two_sided(sample, theta, estimation_theta, estimation_sigma)
    if alternative is 'less':
        p_value = sps.norm.cdf(statistic)
        left = 1 / np.mean(sample) - sps.norm.ppf(1 - alpha / 2) / (np.sqrt(n) * np.mean(sample))
        right = np.inf
        
        
    return None
    

Проверьте гипотезы с провосторонней и левосторонней альтернативой

In [ ]:
<...>

**Какие выводы вы можете сделать?**

**Вывод:** <...>

Теперь посмотрим на выборку меньших размеров

In [ ]:
sample_cut = [0.11731702, 0.75253036, 0.32918642, 0.22823564, 0.04240622,
        0.04239907, 0.01495969, 0.50280772, 0.22977054, 0.30781252]

Выведите статистику, p-value и доверительный интервал. Какой вывод можно сделать из полученных значений?

In [ ]:
<...>

**Вывод:** <...>

---
# Реальный уровень значимости

Напомним, что реальным уровнем значимости критерия $\mathsf{H}_0 \colon \theta = \theta_0$ vs. $\mathsf{H}_1 \colon \theta \neq \theta_0$ называется $\mathsf{P}_{\theta_0}(X \in S)$. Поскольку критерий Вальда асимптотический, при фиксированных $n$ он может отличаться от $\alpha$. В таком случае бывает полезно экспериментально определить реальный уровень значимости с помощью метода Монте-Карло.

С помощью статистических методов можно проверить человека на наличие экстрасенсорных способностей: предложим ему угадать последовательность, состоящую из двух цветов, длины 100. 

Сформулируем задачу на статистическом языке: 

$X_1...X_{100}$ &mdash; выборка из распределения $Bern(p)$

$p=0.5$ отвечает случайному угадыванию.

Cгенерируйте эту выборку.

In [ ]:
sample_size = 100
sample = <...>

Проверьте гипотезу: $\mathsf{H}_0 \colon p=0.5$ vs $\mathsf{H}_1 \colon p \neq 0.5$. Используйте критерий Вальда.

В качестве асимптотически нормальной оценки можно использовать $\widehat{p} = \overline{X}$ с асимптотической дисперсией $\sigma^2(p) = p (1 - p)$.

**Выпишите состоятельную оценку дисперсии и статистику критерия Вальда**


**Ответ:** <...>

Примените критерий, используйте реализованную вами функцию.

In [ ]:
alpha = 0.05
theta = 0.5
estimation_theta = <...>
estimation_sigma = <...>

wald_test(sample, theta, estimation_theta, estimation_sigma)

**Какой вывод вы можете сделать?**

**Ответ:** <...>

Проведи подобный эксперимент 1000 раз и посчитайте долю случаев отвержения гипотезы для определения реального уровня значимости критерия. 

*Замечание:* поскольку погрешность определения реального уровня значимости оценивается как $\frac{1}{\sqrt{k}}$, где $k$ - количество экспериментов, при 1000 повторений точность определения значения составляет $\approx 0.03$ и не является высокой. Такое значение выбрано для скорости подсчета на семинаре, но в реальных экспериментах его стоит брать намного больше.

In [ ]:
samples_count = 1000
sample = <...>

In [ ]:
estimation_theta = <...>
estimation_sigma = <...>

counter = 0
for i in range(samples_count):
    _, p_value, conf_int = wald_test(sample[i], theta, estimation_theta[i], estimation_sigma[i])
    is_rejected = <...>
    if is_rejected:
        counter += 1

counter/samples_count

**Вывод:** <...>